In [1]:
import Pkg
Pkg.add("ITensors")
using ITensors

   Resolving package versions...
  No Changes to `C:\Users\tcsh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\tcsh\.julia\environments\v1.10\Manifest.toml`


In [2]:
N = 3
sites = siteinds("S=1/2",N);

os = OpSum()
for j=1:N-1
  os += -1.0,"X",j,"X",j+1
  os += -1.0,"Y",j,"Y",j+1
  os += +(0.5+2*10),"Z",j,"Z",j+1
  os += -2*10,"Z",j
end
os += -1.0,"X",1,"X",N
os += -1.0,"Y",1,"Y",N
os += +(0.5+2*10),"Z",1,"Z",N
os += -2*10,"Z",N

os += +4*10,"I",1

H = MPO(os,sites);

nsweeps = 10 # number of sweeps is 5
maxdim = [10,20,100,100,200] # gradually increase states kept
cutoff = [1E-10] # desired truncation error

psi0 = randomMPS(sites,2)

energy, psi = dmrg(H,psi0; nsweeps, maxdim, cutoff)

After sweep 1 energy=-4.488962248184456  maxlinkdim=2 maxerr=1.19E-16 time=18.488
After sweep 2 energy=-4.4999998548657665  maxlinkdim=2 maxerr=1.05E-15 time=0.004
After sweep 3 energy=-4.499999999998068  maxlinkdim=2 maxerr=2.22E-16 time=0.003
After sweep 4 energy=-4.499999999999997  maxlinkdim=2 maxerr=5.55E-17 time=0.004
After sweep 5 energy=-4.499999999999996  maxlinkdim=2 maxerr=1.11E-16 time=0.003
After sweep 6 energy=-4.4999999999999964  maxlinkdim=2 maxerr=5.55E-17 time=0.004
After sweep 7 energy=-4.4999999999999964  maxlinkdim=2 maxerr=2.22E-16 time=0.026
After sweep 8 energy=-4.499999999999997  maxlinkdim=2 maxerr=1.67E-16 time=0.003
After sweep 9 energy=-4.499999999999996  maxlinkdim=2 maxerr=6.11E-16 time=0.001
After sweep 10 energy=-4.4999999999999964  maxlinkdim=2 maxerr=2.78E-16 time=0.002


(-4.4999999999999964, MPS
[1] ((dim=2|id=414|"Link,l=1"), (dim=2|id=559|"S=1/2,Site,n=1"))
[2] ((dim=2|id=890|"Link,l=2"), (dim=2|id=733|"S=1/2,Site,n=2"), (dim=2|id=414|"Link,l=1"))
[3] ((dim=2|id=865|"S=1/2,Site,n=3"), (dim=2|id=890|"Link,l=2"))
)

In [29]:
# This is well done!!!
contraction = psi[1] * psi[2] * psi[3];
state = array(contraction, inds(contraction));
state = reshape(state, 8);
threshold = 1e-14;
state .= ifelse.(abs.(state) .< threshold, 0.0, state);
@show state

state = ComplexF64[0.0 + 0.0im, 0.5773502691896253 + 0.0im, 0.5773502691896261 + 0.0im, 0.0 + 0.0im, 0.5773502691896261 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im]


8-element Vector{ComplexF64}:
                0.0 + 0.0im
 0.5773502691896253 + 0.0im
 0.5773502691896261 + 0.0im
                0.0 + 0.0im
 0.5773502691896261 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im

In [64]:
# Now, I have to construct the "probability MPS"

# Here I define the copy tensor
dim = 2
copy_array = zeros(Float64, (dim, dim, dim))

for val in 1:dim
    copy_array[val, val, val] = 1.0
end

# Here I define the "probablity MPS"
psi_prime = prime(psi);

siteinds_psi = siteinds(psi); siteinds_psi_prime = siteinds(psi_prime);

i1 = Index(2, "n=1,p");
i2 = Index(2, "n=2,p");
i3 = Index(2, "n=3,p");

T1 = ITensor(copy_array, siteinds_psi[1], siteinds_psi_prime[1], i1);
T2 = ITensor(copy_array, siteinds_psi[2], siteinds_psi_prime[2], i2);
T3 = ITensor(copy_array, siteinds_psi[3], siteinds_psi_prime[3], i3);

probability_psi = psi[1] * psi[2] * psi[3] * psi_prime[1] * psi_prime[2] * psi_prime[3] * T1 * T2 * T3;

probability_state = array(probability_psi, inds(probability_psi));
probability_state = reshape(probability_state, 8);
threshold = 1e-14;
probability_state .= ifelse.(abs.(probability_state) .< threshold, 0.0, probability_state);
@show probability_state

probability_state = ComplexF64[0.0 + 0.0im, 0.33333333333333276 + 0.0im, 0.3333333333333337 + 0.0im, 0.0 + 0.0im, 0.3333333333333337 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im]


8-element Vector{ComplexF64}:
                 0.0 + 0.0im
 0.33333333333333276 + 0.0im
  0.3333333333333337 + 0.0im
                 0.0 + 0.0im
  0.3333333333333337 + 0.0im
                 0.0 + 0.0im
                 0.0 + 0.0im
                 0.0 + 0.0im

In [65]:
psi_MPO = MPO(N)

# We contract each tensor of the MPS with the copy tensor
@



MPO
#undef
#undef
#undef
